In [1]:
import numpy as np 


# Multi layer perceptron for regression

In [ ]:
class my_MLP():
    def __init__(self,architecture):
        """
        :param architecture: list [n1,n2,n3] corresponding to number of neurons per layer
        """
        self.nb_layer=len(architecture)
        self.weights,self.bias = {},{}
        self.architecture=architecture

        pass

    def initialization(self,data,y):
        weights={}
        bias={}
        all_dimension=[data.shape[1]]+self.architecture+[len(np.unique(y))]

        for layer in range(len(all_dimension)-1):

            weights[f'w{layer+1}']=np.random.randn(all_dimension[layer],all_dimension[layer+1])*np.sqrt(2/all_dimension[layer])
            bias[f'b{layer+1}']=np.random.randn(1,all_dimension[layer+1])
        
        self.weights,self.bias = weights,bias

        print("Initialized layers:", all_dimension)

    def activation(self,z):
        return 1 / (1 + np.exp(-z))
    
    def activation_derivative(self,z):
        s = self.activation(z)
        return s * (1 - s)
    
    def softmax(self,logits): 
        logits = logits - np.max(logits,axis=1,keepdims=True)

        return np.exp(logits)/np.sum(np.exp(logits),axis=1,keepdims=True)

    def loss(self,logits,target): # Cross entropy loss for classification 
        N = len(target)
        probas=self.softmax(logits)
        log_likelihood= -np.log(probas[range(N), target])
        return log_likelihood.mean(), probas


    def forward(self,data):
        self.cache={} # Store activation and output values for backprop
        nb_layers=len(self.architecture)+1 #hidden layer+output
        next_input=data
        self.cache["A0"]=next_input

        for layer in range(1,nb_layers+1):
            w=self.weights[f"w{layer}"]
            b=self.bias[f"b{layer}"]

            
            next_input=next_input@w + b
            self.cache[f"Z{layer}"] = next_input
            

            if layer<nb_layers:
                next_input=self.activation(next_input)
                self.cache[f"A{layer}"]=next_input
        
        self.cache[f"A{nb_layers}"]=next_input

        return next_input
    

    def backward_prop(self,logits,target):
        grads_w={}
        grads_b={}
        nb_layers=len(self.architecture)+1
        probas=self.softmax(logits)
        
        # gradient of loss wrt to logits in last layer
        dZ=probas.copy()
        dZ[range(len(target)),target]-=1
        dZ/=len(target) 
        
        for layer in reversed(range(1,nb_layers+1)):

            #Compute gradient using explicit formula for sigmoid and linearity
            A_prev=self.cache[f"A{layer-1}"] 
            W=self.weights[f"w{layer}"]

            dw=A_prev.T@dZ
            db=np.sum(dZ,axis=0,keepdims=True)

            grads_w[f"w{layer}"]=dw
            grads_b[f"b{layer}"]=db

            dA_prev=dZ@W.T
            if layer>1:
                dZ=dA_prev*self.activation_derivative(self.cache[f"Z{layer-1}"])


        return grads_w,grads_b
    
    def gradient_descent(self,grads_w,grads_b,lr):

        for key in self.weights:
            self.weights[key]-= lr*grads_w[key]

        for key in self.bias:
            self.bias[key]-= lr*grads_b[key]

    def fit(self,data,target,epoch=100,lr=0.01,batch_size=32):
        loss_history=[]

        for _ in range(epoch):
            perm=np.random.permutation(len(data))

            for start in range(0,len(data),batch_size):
                end=start+batch_size
                index=perm[start:end]

                batch_data=data[index]
                batch_target=target[index]
                
                logits=self.forward(batch_data)
                loss,_ =self.loss(logits,batch_target)
                loss_history.append(loss)

                grads_w,grads_b=self.backward_prop(logits,batch_target)
                self.gradient_descent(grads_w,grads_b,lr=lr)

        return loss_history
    
    def predict(self,test_data):

        logits=self.forward(test_data)

        probas=self.softmax(logits)
        prediction=np.argmax(probas,axis=1)

        return prediction
    
    def accuracy(self, test_data,target):
        prediction=self.predict(test_data)

        return np.mean(prediction==target)


## Test on MINST data

In [7]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml("mnist_784", version=1, as_frame=False)
X = mnist["data"]
y = mnist["target"].astype(np.int64)


In [8]:
# preprocessing: normalize and split
X=X/255.0
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [21]:
mlp=my_MLP(architecture=[64,64])
mlp.initialization(X_train,y_train)

loss_history=mlp.fit(X_train,y_train,epoch=50,lr=0.1)

Initialized layers: [784, 64, 64, 10]


In [22]:
acc_train = mlp.accuracy(X_train, y_train)
acc_test = mlp.accuracy(X_test, y_test)

print("Train accuracy:", acc_train)
print("Test accuracy:", acc_test)


Train accuracy: 0.9986071428571428
Test accuracy: 0.9709285714285715


# Pytorch comparison

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class torch_MLP(nn.Module):
    def __init__(self, input_dim, architecture, output_dim):
        super().__init__()
        dims = [input_dim] + architecture + [output_dim]
        self.layers = nn.ModuleList([nn.Linear(dims[i], dims[i+1]) for i in range(len(dims)-1)])

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < len(self.layers) - 1:
                x = torch.sigmoid(x)
        return x

    def fit(self, X, y, epochs=50, lr=0.1):
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        
        opt = torch.optim.Adam(self.parameters(), lr=lr)
        loss_fn = nn.CrossEntropyLoss()

        for e in range(epochs):
            opt.zero_grad()
            loss = loss_fn(self.forward(X), y)
            loss.backward()
            opt.step()

    def predict(self, X):
        X = torch.tensor(X, dtype=torch.float32)
        return torch.argmax(torch.softmax(self.forward(X), dim=1), dim=1).numpy()

    def accuracy(self, X, y):
        return (self.predict(X) == y).mean()


In [36]:
torch_mlp = torch_MLP(input_dim=784, architecture=[64, 64], output_dim=10)

torch_mlp.fit(X_train, y_train, epochs=50, lr=0.001)

print("Train accuracy:", torch_mlp.accuracy(X_train, y_train))
print("Test accuracy:", torch_mlp.accuracy(X_test, y_test))


Train accuracy: 0.550875
Test accuracy: 0.5557142857142857


(56000, 784)